In [3]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import json
import multiprocessing as mp

In [4]:
# from google.colab import files
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My\ Drive

In [5]:
class MVSampler():
    def __init__(self, reference, inference, target_cols):
        """
        reference : pd.DataFrame. 모집단으로 사용될 set.(test를 넣으면 됨)
        inference : pd.DataFrame. 배경집단으로 사용된 inference set
        target_cols : array-like. 해당 column들에 대해서만 확률 계산
        """
        self._pdf = {}
        self._cnt = np.zeros(reference.shape[0])
        self.reference = reference.copy()
        del reference
        self.inference = inference.copy()
        del inference
        self.target_cols = target_cols

        bins = int(1+3.322*np.log10(self.inference.shape[0]))
        for col in self.target_cols:
            p, bin_edge = np.histogram(self.inference[col].values,
                                       bins=np.min([bins, len(self.inference[col].unique())]),
                                       density=True)
            self._pdf[col] = (p, bin_edge)

        for col in self.target_cols:
            bin_edge = self._pdf[col][1]
            b = (bin_edge[1:] + bin_edge[:-1])/2
            lcond = self.reference[col].values>=b.min()
            rcond = self.reference[col].values<=b.max()
            self._cnt += (lcond*rcond*1)

    def build(self, row_min = -1, outlier = 0):
        """
        row_min : int. sampling에 사용될 reference의 최소 row 갯수
        outlier : int. ref/inf 의 target_col 분포를 비교할 때, 고려하지 않을 column의 갯수
        모든 target_col이 ref/inf 분포가 겹치지 않을 수 있기 때문에 일정 row 갯수가 확보될 때 까지 증가
        """
        if row_min  < 0:
            row_min = int(self.inference.shape[0] * 0.1)

        nsize = len(np.where(self._cnt>=len(self.target_cols)-outlier)[0])
        while (nsize <= row_min) and (outlier <= len(self.target_cols)):
            outlier += 1
            nsize = len(np.where(self._cnt>=len(self.target_cols)-outlier)[0])
        self._outlier = outlier

        self.log_likelihood = np.zeros(self.reference.shape[0])
        for i in range(self.reference.shape[0]):
            row_vector = self.reference.iloc[i, :]
            if self._cnt[i]>=len(self.target_cols)-outlier:
                p = np.array([self.interp_pdf(self._pdf[col], row_vector[col]) for col in self.target_cols])
                self.log_likelihood[i] = np.sum(np.log10(p[np.where(p!=0)[0]]))
                # np.power(10, np.sum(np.log10(p[np.where(p!=0)[0]])))

    @staticmethod
    def interp_pdf(pdf, x):
        p, bin_edge = pdf
        b = (bin_edge[1:] + bin_edge[:-1])/2
        if x<=b.min() or x>=b.max():
            return 0
        f = interp1d(b, p, kind="linear")
        y = f(x)
        return y

    def sample(self, n=-1, seed=None, return_index=False):
        """
        n : int. sample의 크기
        return_index : bool. True일 경우 sample의 index를 array로 return, False일 경우 sample을 pd.DataFrame으로 return. 기본은 False
        return : pd.DataFrame / array-like
        """
        n = n if n>0 else self.inference.shape[0]//10
        prob = np.power(10, self.log_likelihood)
        prob /= np.sum(prob)
        rng = np.random.default_rng(seed)
        idx = rng.choice(np.arange(0, self.reference.shape[0]), replace=True, p=prob, size=n)
        if return_index:
            return idx
        return self.reference.iloc[idx, :]

In [6]:
def dist(slope, intercept, x, y):
    return abs(slope*x-y+intercept)/np.sqrt(slope**2+1)
def linear(x, a, b):
    return a*x+b
def chi2(slope, intercept, x, y):
    dobs = dist(slope, intercept, x, y)**2
    return np.sum(dobs)/ 2 / (len(x)-2)


class PerformanceEstimator():
    def __init__(self, drift, drift_smp, metric_smp):
        self.drift = drift
        self.drift_smp = drift_smp
        self.metric_smp = metric_smp

    def estimate(self, n = 5):
        drift_list = self.drift_smp.keys()
        metric_list = self.metric_smp.keys()
        if len(self.drift[list(self.drift.keys())[0]])<n:
            return self.metric_smp

        metric_est = {}
        for key, m in self.metric_smp.items():
            metric_est[key] = self.correction(metric= m, drift =self.drift, drift_smp = self.drift_smp)
        return metric_est

    @staticmethod
    def nlinearfit(x, y, th = 2):
        x = np.array(x) if isinstance(x, list) else x
        y = np.array(y) if isinstance(y, list) else y
        m = (y.max() - y.min()) / (x.min() - x.max())
        i = y.mean() - m * x.mean()
        d, c2 = dist(m, i, x, y), chi2(m, i, x, y)
        cnt = 0

        while c2>th and m>0 and cnt<len(x)-4:
            xn, yn = x[np.argsort(d)[:len(x)-cnt]], y[np.argsort(d)[:len(x)-cnt]]
            popt, _ = curve_fit(linear, xn, yn)
            m, i = popt[0], popt[1]
            c2 = chi2(m, i, xn, yn)
            cnt += 1
        c2 = chi2(m, i, x, y)
        return m, i, c2

    @staticmethod
    def correction(metric, drift, drift_smp):
        metric_cor = 0
        denominator = 0

        for k in drift_smp.keys():
            d, d_smp =  drift[k], drift_smp[k]
            dbar = d[-1] - np.min(d)
            dbar_smp = d_smp - np.min(d_smp)
            r1, beta, chi2 = PerformanceEstimator.nlinearfit(dbar_smp, metric, 2)
            metric_cor += (r1 * dbar + beta) / chi2
            denominator +=  (1/chi2)

        return metric_cor/denominator

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
def accuracy(args):
    model, X, y = args[0], args[1], args[2]
    return accuracy_score(model.predict(X), y)*100

def f1(args):
    model, X, y = args[0], args[1], args[2]
    return f1_score(model.predict(X), y) *100

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def D3(args):
    reference, inference, model = args[0], args[1], args[2]
    y1 = np.zeros(reference.shape[0]).reshape(-1, 1)
    y2 = np.ones(inference.shape[0]).reshape(-1, 1)
    X = np.vstack((reference, inference))
    y = np.vstack((y1, y2))
    lr = LogisticRegression(solver="liblinear", random_state=0, max_iter=5000).fit(X, y.ravel())
    return roc_auc_score(y, lr.predict_proba(X)[:, 1])

from scipy.stats import ks_2samp, entropy
def uncertainty(args):
    reference, inference, model = args[0], args[1], args[2]
    entropy_ref = entropy(model.predict_proba(reference), axis = -1)
    entropy_inf = entropy(model.predict_proba(inference), axis = -1)
    d, p = ks_2samp(entropy_ref, entropy_inf)
    return d

from scipy.linalg import sqrtm
def FID(reference, inference, model):
    act_ref, act_inf = [], []

    for i in range(reference.shape[0]):
        refr, infr = reference[i, :], inference[i, :]
        vmin, vmax = min(refr.min(), infr.min()), max(refr.max(), infr.max())
        bins = int(1+3.322*np.log10(max(len(refr), len(infr))))
        refh, _ = np.histogram(refr, range=(vmin, vmax), bins=bins)
        infh, _ = np.histogram(infr, range=(vmin, vmax), bins=bins)
        act_ref.append(refh)
        act_inf.append(infh)

    act_ref = np.array(act_ref)
    act_inf = np.array(act_inf)
    mu1, sigma1 = np.mean(act_ref, axis=0), np.cov(act_ref, rowvar=False)
    mu2, sigma2 = np.mean(act_inf, axis=0), np.cov(act_inf, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    mat_dot = sigma1.dot(sigma2)
    covmean = sqrtm(mat_dot)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [22]:
from sklearn.ensemble import RandomForestClassifier

def run(df_train, df_test, df_live, target_cols, drop_cols, y, n_proc):

    drift_checkers = [D3, uncertainty]
    drift_list = [dc.__name__ for dc in drift_checkers]
    metrics = [accuracy]
    metric_list = [m.__name__ for m in metrics]


    DB = {}
    for metric in metrics:
        DB[metric.__name__] = []
        DB[metric.__name__+"_smp"] = []
        DB[metric.__name__+"_est"] = []
    for dc in drift_checkers:
        DB[dc.__name__] = []
        DB[dc.__name__+"_smp"] = []
    DB["index"] = []

    X_train = df_train.drop(columns=drop_cols).values
    y_train = df_train[y].values

    model = RandomForestClassifier(max_depth=6).fit(X_train, y_train)

    X_test = df_test.drop(columns=drop_cols).values
    y_test = df_test[y].values



    for dc in drift_checkers:
        v = dc([X_train, X_test, model])
        DB[dc.__name__].append(v)
        DB[dc.__name__+"_smp"].append(v)
    for m in metrics:
        v = m([model, X_test, y_test])
        DB[m.__name__].append(v)
        DB[m.__name__+"_smp"].append(v)
        DB[m.__name__+"_est"].append(v)
    DB["index"].append(-1)

    print("DB setup")
    for i, live in enumerate(df_live):
        if live.shape[0]>0:
            DB["index"].append(i)
            Xlive = live.drop(columns=drop_cols).values
            ylive = live[y].values

            for dc in drift_checkers:
                DB[dc.__name__].append(dc([X_train, Xlive, model]))

            mvs = MVSampler(df_test, live, target_cols)
            mvs.build(0)
            sample = [mvs.sample(return_index=False) for _ in range(100)]

            args_drift = [(X_train, sample[i].drop(columns=drop_cols).values, model) for i in range(100)]
            for dc in drift_checkers:
                if __name__=="__main__":
                    with mp.Pool(8) as pool:
                        res = pool.map(dc, args_drift)
                DB[dc.__name__+"_smp"].append(np.mean(res))

            args_metric = [(model, sample[i].drop(columns=drop_cols).values, sample[i][y].values) for i in range(100)]
            for m in metrics:
                if __name__=="__main__":
                    with mp.Pool(n_proc) as pool:
                        res = pool.map(m, args_metric)
                DB[m.__name__+"_smp"].append(np.mean(res))


            ### real metric
            for m in metrics:
                DB[m.__name__].append(m([model, Xlive, ylive]))

            ### DB에서 load
            DB_drift = {k:DB[k] for k in drift_list}
            DB_drift_smp = {k:DB[k+"_smp"] for k in drift_list}
            DB_metric_smp = {k:DB[k+"_smp"] for k in metric_list}

            pe = PerformanceEstimator(
            drift = DB_drift,
            drift_smp = DB_drift_smp,
            metric_smp = DB_metric_smp)

            metric_est = pe.estimate(n = 5)
            for k, v in metric_est.items():
                if isinstance(v, list):
                    DB[k+"_est"].append(v[-1])
                else:
                    DB[k+"_est"].append(v)
    return DB


In [9]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [10]:
data_size = 40000
cor = [1.5, 1.2, 0.9 , 0.7, 0.5]
y = (np.random.normal(0.5,0.5,100) > 0.5).astype(int)
y = (np.random.normal(0.5,0.5,data_size) > 0.5).astype(int)
x = np.zeros((data_size,5))
for ix,cor_ in enumerate(cor):
    x[:,ix] = y + np.random.normal(1,cor_,data_size)
for i in range(400):
    s = i*100
    x[s:s+100,4] = y[s:s+100] + np.random.normal(1,0.5+(i/100),100)
columns=['x0','x1','x2','x3','z0']
df = pd.DataFrame(x,columns=columns)
df["y"] = y

### Reference =split into=> train/test ###
size = 5000
df_train = df.iloc[:size, :]
df_test = df.iloc[size:2*size, :]

### Inference =split into=> indiv live set ###
inference = df.iloc[2*size:, :]
s = 1000
live_set = [inference.iloc[i*s:(i+1)*s, :] for i in range(inference.shape[0]//s)]

### Target/Drop/Y columns ###
# target_cols : sampling 할 때 고려할 column list. feature importance 등을 사용하여 설정
# drop_cols : X domain에서 제외할 column명
# ylabel : y domain column명

ylabel = "y"
target_cols = df_train.drop(columns=ylabel).columns
drop_cols = ["y"]

In [11]:
# target_cols = np.genfromtxt("/content/drive/My Drive/kaggle/loan.txt", delimiter='\t', dtype=str)
# df = pd.read_parquet("/content/drive/My Drive/kaggle/loan.parquet")

# from astropy.time import Time
# df = df[(df.time>=Time("2013-01-01").mjd) & (df.time<Time("2018-01-01").mjd)]
# dt = 30
# df_train = df[df.time<Time("2014-01-01").mjd]
# df_test = df[(df.time>=Time("2014-01-01").mjd) & (df.time<Time("2015-01-01").mjd)]
# live = [df[(df.time>=Time("2015-01-01").mjd+i*dt) & (df.time<Time("2015-01-01").mjd+(i+1)*dt)] for i in range(int((df.time.max()-Time("2015-01-01").mjd)//dt))]
# drop_cols = ["time", "loan_paid"]
# y = "loan_paid"

In [23]:
DB = run(df_train, df_test, live_set, target_cols, drop_cols, ylabel, 8)

DB setup


In [ ]:
plt.figure(figsize=(6, 4))
gs = GridSpec(nrows=2, ncols=1, hspace=0, wspace=0)
name = "accuracy"
axes = [plt.subplot(gs[i]) for i in range(2)]
axes[0].plot(DB["index"], DB[name], label=name+'_ori')
axes[0].plot(DB["index"], DB[name+"_smp"], label=name+"_smp")
axes[0].plot(DB["index"], DB[name+"_est"], label=name+"_est", linestyle="--")

axes[1].plot(DB["index"], DB["uncertainty"], label="unc")
axes[1].plot(DB["index"], DB["uncertainty_smp"], label="smp")

for ax in axes:
    ax.set_xlim(min(DB["index"]), max(DB["index"]))
    ax.grid()
    ax.legend(loc='best')
plt.show()